In [6]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.models import inception_v3
from scipy.stats import entropy
import numpy as np
import os
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data
from PIL import Image
from torchvision.models.inception import inception_v3

import numpy as np
from scipy.stats import entropy

def inception_score(imgs, cuda=True, batch_size=32, resize=False, splits=1):
    N = len(imgs)

    assert batch_size > 0
    assert N > batch_size

    # Set up dtype
    if cuda:
        dtype = torch.cuda.FloatTensor
    else:
        if torch.cuda.is_available():
            print("WARNING: You have a CUDA device, so you should probably set cuda=True")
        dtype = torch.FloatTensor

    # Set up dataloader
    dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)

    # Load inception model
    inception_model = inception_v3(pretrained=True, transform_input=False).type(dtype)
    inception_model.eval();
    up = nn.Upsample(size=(299, 299), mode='bilinear').type(dtype)
    def get_pred(x):
        if resize:
            x = up(x)
        x = inception_model(x)
        return F.softmax(x).data.cpu().numpy()

    # Get predictions
    preds = np.zeros((N, 1000))

    for i, batch in enumerate(dataloader, 0):
        batch = batch.type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]

        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batchv)

    # Now compute the mean kl-div
    split_scores = []

    for k in range(splits):
        part = preds[k * (N // splits): (k+1) * (N // splits), :]
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))

    return np.mean(split_scores), np.std(split_scores)
# Assuming images is a PyTorch tensor of shape [batch_size, 3, 768, 768]
# Make sure to convert it to a numpy array before using the function
dataset_root = './outputs/phrases2/'
generated_images = []

files = os.listdir(dataset_root)
for file in files:
    file_path = os.path.join(dataset_root, file)
    obj_files = os.listdir(file_path)
    for obj_file in obj_files:
        if 'SynGen_LCM' in obj_file:
            obj_file_path = os.path.join(file_path, obj_file)
            generated_images.append(obj_file_path)

# Example usage
mean = []
std = []
a=[]
for generated_image in generated_images:
    if generated_image.endswith('.png'):
        generated_image = np.array(Image.open(generated_image))
        a.append(generated_image)
a=np.array(a)

transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])        
generated_image = torch.tensor(a).permute(0,3,1,2)
#data_loader = DataLoader(generated_image, batch_size=64, shuffle=False)
is_score_mean, is_score_std = inception_score(generated_image, cuda=True, batch_size=32, resize=True, splits=10)


print(f"Inception Score: {is_score_mean}, Inception std:{is_score_std}")

/home/jeeit17/.conda/envs/mm/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jeeit17/.conda/envs/mm/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_32276/3338306929.py:45: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x).data.cpu().numpy()


Inception Score: 3.752881337392322, Inception std:0.749630610926113


/bin/bash: gdown: command not found
